In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 84.8MB/s]


In [2]:
!ls

happy-or-sad.zip  h-or-s  sample_data


In [11]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model(optimizer, layer_list, epochs):
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        
        # YOUR CODE START HERE
        def on_epoch_end(self, epoch, logs={}):
          if(logs.get('accuracy') > DESIRED_ACCURACY):
            print('\nStopping Training! Reached 99.9% accuracy!')
            self.model.stop_training = True

    callback = myCallback()
        # YOUR CODE END HERE
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential(layer_list)

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255.)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory('h-or-s/',
                                                        target_size=(150, 150),
                                                        batch_size=10,
                                                        class_mode='binary'
                                                       )
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(train_generator,
                        steps_per_epoch=2,
                        epochs=epochs,
                        callbacks=[callback],
                        verbose = 1
                       )
    
    return history.history['accuracy'][-1]

In [4]:
train_happy_sad_model(
    optimizer = 'Adam',
    layer_list = [
          tf.keras.layers.Conv2D(512, (3,3), activation='relu', input_shape=(150, 150, 3)),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

Found 80 images belonging to 2 classes.
Epoch 1/20
2/2 [==============================] - 6s 172ms/step - loss: 0.6916 - accuracy: 0.4500
Epoch 2/20
2/2 [==============================] - 0s 172ms/step - loss: 0.6812 - accuracy: 0.6000
Epoch 3/20
2/2 [==============================] - 0s 165ms/step - loss: 0.6617 - accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 0s 173ms/step - loss: 0.7035 - accuracy: 0.4500
Epoch 5/20
2/2 [==============================] - 0s 178ms/step - loss: 0.5409 - accuracy: 0.8000
Epoch 6/20
2/2 [==============================] - 0s 167ms/step - loss: 0.5058 - accuracy: 0.6500
Epoch 7/20
2/2 [==============================] - 0s 168ms/step - loss: 0.4601 - accuracy: 0.6000
Epoch 8/20
2/2 [==============================] - 0s 177ms/step - loss: 0.4618 - accuracy: 0.8000
Epoch 9/20
2/2 [==============================] - 0s 167ms/step - loss: 0.2841 - accuracy: 0.9000
Epoch 10/20
2/2 [==============================] - 0s 166ms/step - loss: 0.348

1.0

In [10]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(512, (3,3), activation='relu', input_shape=(150, 150, 3)),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

Found 80 images belonging to 2 classes.
Epoch 1/20
2/2 [==============================] - 2s 168ms/step - loss: 2.5166 - accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 0s 168ms/step - loss: 0.7338 - accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 0s 168ms/step - loss: 0.6742 - accuracy: 0.5500
Epoch 4/20
2/2 [==============================] - 0s 170ms/step - loss: 1.2968 - accuracy: 0.5500
Epoch 5/20
2/2 [==============================] - 0s 176ms/step - loss: 0.7118 - accuracy: 0.4500
Epoch 6/20
2/2 [==============================] - 0s 172ms/step - loss: 0.6950 - accuracy: 0.5500
Epoch 7/20
2/2 [==============================] - 0s 164ms/step - loss: 0.6984 - accuracy: 0.2500
Epoch 8/20
2/2 [==============================] - 0s 170ms/step - loss: 0.6941 - accuracy: 0.3500
Epoch 9/20
2/2 [==============================] - 0s 166ms/step - loss: 0.6847 - accuracy: 0.6500
Epoch 10/20
2/2 [==============================] - 0s 167ms/step - loss: 0.700

0.6000000238418579

In [14]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(512, (3,3), activation='relu', input_shape=(150, 150, 3)),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 2s 204ms/step - loss: 1.3744 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 185ms/step - loss: 0.8654 - accuracy: 0.4500
Epoch 3/50
2/2 [==============================] - 0s 183ms/step - loss: 0.6781 - accuracy: 0.7000
Epoch 4/50
2/2 [==============================] - 0s 187ms/step - loss: 0.6995 - accuracy: 0.5500
Epoch 5/50
2/2 [==============================] - 0s 185ms/step - loss: 0.6946 - accuracy: 0.4000
Epoch 6/50
2/2 [==============================] - 0s 176ms/step - loss: 0.7017 - accuracy: 0.4000
Epoch 7/50
2/2 [==============================] - 0s 183ms/step - loss: 0.6577 - accuracy: 0.6000
Epoch 8/50
2/2 [==============================] - 0s 180ms/step - loss: 0.5781 - accuracy: 0.7000
Epoch 9/50
2/2 [==============================] - 0s 185ms/step - loss: 1.1309 - accuracy: 0.3500
Epoch 10/50
2/2 [==============================] - 0s 184ms/step - loss: 0.686

1.0

In [15]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(150, 150, 3)),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 2s 89ms/step - loss: 4.0994 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 86ms/step - loss: 0.7296 - accuracy: 0.4500
Epoch 3/50
2/2 [==============================] - 0s 87ms/step - loss: 0.7149 - accuracy: 0.3500
Epoch 4/50
2/2 [==============================] - 0s 83ms/step - loss: 0.6947 - accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 0s 87ms/step - loss: 0.6382 - accuracy: 0.8000
Epoch 6/50
2/2 [==============================] - 0s 80ms/step - loss: 0.9569 - accuracy: 0.3500
Epoch 7/50
2/2 [==============================] - 0s 82ms/step - loss: 0.6120 - accuracy: 0.8000
Epoch 8/50
2/2 [==============================] - 0s 82ms/step - loss: 0.5563 - accuracy: 0.8500
Epoch 9/50
2/2 [==============================] - 0s 84ms/step - loss: 0.6661 - accuracy: 0.5500
Epoch 10/50
2/2 [==============================] - 0s 86ms/step - loss: 0.4948 - accura

1.0

In [16]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 1s 22ms/step - loss: 5.3644 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 31ms/step - loss: 0.9438 - accuracy: 0.4500
Epoch 3/50
2/2 [==============================] - 0s 32ms/step - loss: 0.6380 - accuracy: 0.6500
Epoch 4/50
2/2 [==============================] - 0s 27ms/step - loss: 0.4529 - accuracy: 0.8500
Epoch 5/50
2/2 [==============================] - 0s 33ms/step - loss: 0.4145 - accuracy: 0.7500
Epoch 6/50
2/2 [==============================] - 0s 27ms/step - loss: 0.5297 - accuracy: 0.7000
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.3706 - accuracy: 0.7500
Epoch 8/50
2/2 [==============================] - 0s 29ms/step - loss: 0.2736 - accuracy: 0.8000
Epoch 9/50
2/2 [==============================] - 0s 33ms/step - loss: 0.1665 - accuracy: 0.9000
Epoch 10/50
1/2 [==============>...............] - ETA: 0s - loss: 0.1065 - accuracy: 1

1.0

In [17]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 2s 43ms/step - loss: 4.8229 - accuracy: 0.6000
Epoch 2/50
2/2 [==============================] - 0s 46ms/step - loss: 1.0388 - accuracy: 0.3000
Epoch 3/50
2/2 [==============================] - 0s 40ms/step - loss: 0.6932 - accuracy: 0.4500
Epoch 4/50
2/2 [==============================] - 0s 45ms/step - loss: 0.6931 - accuracy: 0.4500
Epoch 5/50
2/2 [==============================] - 0s 46ms/step - loss: 0.6928 - accuracy: 0.3500
Epoch 6/50
2/2 [==============================] - 0s 43ms/step - loss: 0.6918 - accuracy: 0.6000
Epoch 7/50
2/2 [==============================] - 0s 43ms/step - loss: 0.6894 - accuracy: 0.5000
Epoch 8/50
2/2 [==============================] - 0s 42ms/step - loss: 0.6922 - accuracy: 0.4000
Epoch 9/50
2/2 [==============================] - 0s 46ms/step - loss: 0.6913 - accuracy: 0.6500
Epoch 10/50
2/2 [==============================] - 0s 43ms/step - loss: 0.6868 - accura

1.0

In [18]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 1s 49ms/step - loss: 14.8036 - accuracy: 0.6500
Epoch 2/50
2/2 [==============================] - 0s 47ms/step - loss: 0.6869 - accuracy: 0.6000
Epoch 3/50
2/2 [==============================] - 0s 47ms/step - loss: 0.8986 - accuracy: 0.3500
Epoch 4/50
2/2 [==============================] - 0s 47ms/step - loss: 0.6746 - accuracy: 0.5500
Epoch 5/50
2/2 [==============================] - 0s 53ms/step - loss: 0.5913 - accuracy: 0.5500
Epoch 6/50
2/2 [==============================] - 0s 49ms/step - loss: 0.6562 - accuracy: 0.5000
Epoch 7/50
1/2 [==============>...............] - ETA: 0s - loss: 0.3980 - accuracy: 1.0000
Stopping Training! Reached 99.9% accuracy!
2/2 [==============================] - 0s 46ms/step - loss: 0.3777 - accuracy: 1.0000


1.0

In [19]:
 from tensorflow.keras.optimizers import RMSprop

train_happy_sad_model(
    optimizer = RMSprop(learning_rate=0.001),
    layer_list = [
          tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(16, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')
    ],
    epochs=50
)

Found 80 images belonging to 2 classes.
Epoch 1/50
2/2 [==============================] - 2s 174ms/step - loss: 5.0560 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 171ms/step - loss: 0.8315 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 169ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 172ms/step - loss: 0.6907 - accuracy: 0.7000
Epoch 5/50
2/2 [==============================] - 0s 173ms/step - loss: 0.7201 - accuracy: 0.6000
Epoch 6/50
2/2 [==============================] - 0s 172ms/step - loss: 0.6840 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 166ms/step - loss: 0.6493 - accuracy: 0.5500
Epoch 8/50
2/2 [==============================] - 0s 171ms/step - loss: 0.7054 - accuracy: 0.3000
Epoch 9/50
2/2 [==============================] - 0s 172ms/step - loss: 0.5028 - accuracy: 0.9000
Epoch 10/50
2/2 [==============================] - 0s 170ms/step - loss: 0.449

1.0